In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#%matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
events_data = pd.read_csv('../input/event-data-train/event_data_train.csv')
submission_data = pd.read_csv('../input/submission-data-train/submissions_data_train.csv')

In [ ]:
events_data.head(10)

In [ ]:
submission_data.head(10)

In [ ]:
events_data['action'].unique()

In [ ]:
events_data['date'] = pd.to_datetime(events_data.timestamp, unit='s')

In [ ]:
submission_data['date'] = pd.to_datetime(submission_data.timestamp, unit='s')

In [ ]:
events_data.head(10)

In [ ]:
submission_data.head(10)

In [ ]:
wrong = submission_data[submission_data.submission_status == 'wrong']
wrong_user = wrong.groupby('step_id', as_index=False).agg({'user_id':'sum'})
df_sorted = wrong_user.sort_values("user_id", ascending=False)
df_sorted

In [ ]:
events_data.dtypes

In [ ]:
events_data['day'] = events_data.date.dt.date

In [ ]:
submission_data['day'] = submission_data.date.dt.date

In [ ]:
events_data.head(10)

In [ ]:
submission_data.head(10)

In [ ]:
events_data.groupby('day').user_id.nunique().plot(figsize=(15,8))

In [ ]:
df = events_data.groupby('day', as_index=False).aggregate({'user_id': 'nunique'}).rename(columns = {'user_id': 'user'})
df

In [ ]:
plt.close ()

In [ ]:
df.plot(figsize=(15,8)) 

In [ ]:
print(df['user'].dtypes)
print(df['day'].dtypes)

In [ ]:
events_data.action.hist()

In [ ]:
events_data[events_data.action == 'passed'].groupby('user_id', as_index=False).agg({'step_id':'count'}).rename(columns={'step_id':'passed_steps'}).passed_steps.hist()

In [ ]:
users_events_data = events_data.pivot_table(index='user_id', columns='action', values='step_id', aggfunc='count', fill_value=0).reset_index()
users_events_data

In [ ]:
users_scores = submission_data.pivot_table(index='user_id', columns='submission_status', values='step_id', aggfunc='count', fill_value=0).reset_index()

In [ ]:
users_scores.head(10)

In [ ]:
res1 = users_scores.loc[[(users_scores['correct']).idxmax()]]
res1

In [ ]:
subset_1 = users_scores[(users_scores.correct > 400)]
subset_1

In [ ]:
users_events_data.hist(figsize=(10, 6))

In [ ]:
events_data[['user_id', 'day', 'timestamp']].drop_duplicates(subset=['user_id', 'day']).groupby('user_id')['timestamp'].apply(list).head()

In [ ]:
events_data[['user_id', 'day', 'timestamp']].drop_duplicates(subset=['user_id', 'day']).groupby('user_id')['timestamp'].apply(list).apply(np.diff).head()

In [ ]:
gap_data = events_data[['user_id', 'day', 'timestamp']].drop_duplicates(subset=['user_id', 'day']).groupby('user_id')['timestamp'].apply(list).apply(np.diff).values

In [ ]:
gap_data = pd.Series(np.concatenate(gap_data, axis=0))

In [ ]:
gap_data = gap_data / (24 * 60 * 60)
gap_data

In [ ]:
gap_data[gap_data < 200].hist()

In [ ]:
gap_data.quantile(0.95)

In [ ]:
users_data = events_data.groupby('user_id', as_index=False).agg({'timestamp':'max'}).rename(columns={'timestamp':'last_timestamp'})

In [ ]:
now = events_data.timestamp.max()
drop_out_threshold = 30 * 24 * 60 * 60
print(now, drop_out_threshold)

In [ ]:
users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threshold

In [ ]:
users_data.head()

In [ ]:
users_data = users_data.merge(users_scores, how='outer')
users_data

In [ ]:
users_data = users_data.fillna(0)
users_data

In [ ]:
users_data = users_data.merge(users_events_data, how='outer')
users_data

In [ ]:
users_days = events_data.groupby('user_id').day.nunique().to_frame().reset_index()
users_days

In [ ]:
users_data = users_data.merge(users_days, how='outer')
users_data

In [ ]:
print(users_data.user_id.nunique(), events_data.user_id.nunique(), sep=(' '))

In [ ]:
users_data['passed_course'] = users_data.passed > 170
users_data

In [ ]:
users_data.groupby('passed_course').count()

In [ ]:
100 * 1425 / 17809

In [ ]:
users_data[(users_data.passed_course ==True) &  (users_data.is_gone_user ==False)]

In [ ]:
from sklearn import tree

In [ ]:
import math
E_sh_sob=(1/1)*math.log2((1/1)) - 0
E_sh_kot=-(4/9)*math.log2((4/9)) - (5/9)*math.log2((5/9))
E_gav_sob=0 - (5/5)*math.log2((5/5))
E_gav_kot=-(4/5)*math.log2((4/5)) - (1/5)*math.log2((1/5))
E_laz_sob=0 - (6/6)*math.log2((6/6))
E_laz_kot=-(4/4)*math.log2((4/4)) - 0
print('E_sh_sob=', E_sh_sob, 'E_sh_kot=', E_sh_kot, 'E_gav_sob=', E_gav_sob, 'E_gav_kot=', E_gav_kot, 'E_laz_sob=', E_laz_sob, 'E_laz_kot=', E_laz_kot, sep=' ')

In [ ]:
E = -(4/10)*math.log2((4/10)) - (6/10)*math.log2((6/10))

IG_sh = round((E - (1/10)*E_sh_sob - (9/10)*E_sh_kot), 2)
IG_gav = round((E - (5/10)*E_gav_sob - (5/10)*E_gav_kot),2)
IG_laz = round((E - (6/10)*E_laz_sob - (6/10)*E_laz_kot),2)
print(IG_sh, IG_gav, IG_laz, sep=(' '))

In [ ]:
titanic_data = pd.read_csv('../input/titanic/train.csv')

In [ ]:
titanic_data.head()

In [ ]:
titanic_data.isnull().sum()

In [ ]:
Xt = titanic_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
yt = titanic_data['Survived']

In [ ]:
Xt.head()

In [ ]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [ ]:
Xt = pd.get_dummies(Xt)

In [ ]:
Xt.head()

In [ ]:
Xt.shape

In [ ]:
Xt = Xt.fillna({'Age': Xt.Age.median()})

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_traint, X_testt, y_traint, y_testt = train_test_split(Xt, yt, test_size=0.33, random_state=42)

In [ ]:
clf.fit(Xt, yt)

In [ ]:
clf.fit(X_traint, y_traint)

In [ ]:
clf.score(Xt, yt)

In [ ]:
clf.score(X_traint, y_traint)

In [ ]:
clf.score(X_testt, y_testt)

In [ ]:
scores_data = pd.DataFrame()

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
for max_depth in range(1, 100):
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    clf.fit(X_traint, y_traint)
    train_score = clf.score(X_traint, y_traint)
    test_score = clf.score(X_testt, y_testt)
    mean_cross_val_score = cross_val_score(clf, X_traint, y_traint, cv=5).mean()
    temp_score_data = pd.DataFrame({'max_depth':[max_depth], 'train_score':[train_score], 'test_score':[test_score], 'cross_val_score':[mean_cross_val_score]})
    scores_data = scores_data.append(temp_score_data)

In [ ]:
scores_data.head()

In [ ]:
scores_data_long = pd.melt(scores_data, id_vars =['max_depth'], value_vars =['train_score', 'test_score', 'cross_val_score'], var_name='set_type', value_name='score')

In [ ]:
scores_data_long.head()

In [ ]:
sns.lineplot(x='max_depth', y='score', hue='set_type', data=scores_data_long)

In [ ]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [ ]:
cross_val_score(clf, X_traint, y_traint, cv=5)

In [ ]:
cross_val_score(clf, X_traint, y_traint, cv=5).mean()

In [ ]:
train_iris = pd.read_csv('../input/train-iris/train_iris.csv')
test_iris = pd.read_csv('../input/test-iris/test_iris.csv')

In [ ]:
train_iris

In [ ]:
test_iris.head()

In [ ]:
X_iris = train_iris.drop(['Unnamed: 0', 'species'], axis=1)
y_iris = train_iris['species']
X_test_iris = test_iris.drop(['Unnamed: 0', 'species'], axis=1)
y_test_iris = test_iris['species']

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
scores_data_iris = pd.DataFrame()

In [ ]:
rc =np.random.seed(0)
for max_depth in range(1, 100):
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth, random_state=rc)
    clf.fit(X_iris, y_iris)
    train_score = clf.score(X_iris, y_iris)
    test_score = clf.score(X_test_iris, y_test_iris)
    mean_cross_val_score = cross_val_score(clf, X_iris, y_iris, cv=5).mean()
    temp_score_data = pd.DataFrame({'max_depth':[max_depth], 'train_score':[train_score], 'test_score':[test_score], 'cross_val_score':[mean_cross_val_score]})
    scores_data_iris = scores_data_iris.append(temp_score_data)

In [ ]:
scores_data_iris

In [ ]:
scores_data_long_iris = pd.melt(scores_data_iris, id_vars =['max_depth'], value_vars =['train_score', 'test_score', 'cross_val_score'], var_name='set_type', value_name='score')

In [ ]:
scores_data_long_iris

In [ ]:
sns.lineplot(x='max_depth', y='score', hue='set_type', data=scores_data_long_iris)

In [ ]:
dogs_cats =pd.read_csv('../input/dogs-cats/dogs_n_cats.csv')

In [ ]:
dogs_cats

In [ ]:
dogs_cats.Шерстист.unique()

In [ ]:
X_dogs_cats = dogs_cats.drop(['Вид'], axis=1)
y_dogs_cats = dogs_cats['Вид']


In [ ]:
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_dogs_cats, y_dogs_cats, test_size=0.33, random_state=42)

In [ ]:
scores_data_dogs_cats = pd.DataFrame()
rc =np.random.seed(0)
for max_depth in range(1, 5):
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth, random_state=rc)
    clf.fit(X_train_d, y_train_d)
    train_score = clf.score(X_train_d, y_train_d)
    test_score = clf.score(X_test_d, y_test_d)
    mean_cross_val_score = cross_val_score(clf, X_train_d, y_train_d, cv=5).mean()
    temp_score_data = pd.DataFrame({'max_depth':[max_depth], 'train_score':[train_score], 'test_score':[test_score], 'cross_val_score':[mean_cross_val_score]})
    scores_data_dogs_cats = scores_data_dogs_cats.append(temp_score_data)

In [ ]:
scores_data_dogs_cats

In [ ]:
scores_data_long_dogs_cats = pd.melt(scores_data_dogs_cats, id_vars =['max_depth'], value_vars =['train_score', 'test_score', 'cross_val_score'], var_name='set_type', value_name='score')

In [ ]:
sns.lineplot(x='max_depth', y='score', hue='set_type', data=scores_data_long_dogs_cats)

In [ ]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=rc)

In [ ]:
clf.fit(X_dogs_cats, y_dogs_cats)

In [ ]:
data = pd.read_json('/kaggle/input/new-data/dataset_209691_15.txt')

In [ ]:
data

In [ ]:
rez = clf.predict(data)

In [ ]:
rez = list(rez)
result = {i: rez.count(i) for i in rez}
result


In [ ]:
data = pd.read_json('../input/12345678/dataset_209691_15.txt')
rez = clf.predict(data)
rez = list(rez)
result = {i: rez.count(i) for i in rez}
result

In [ ]:
song= pd.read_csv('../input/songs-data/songs.csv')

In [ ]:
song

In [ ]:
X_song = song.drop(['artist', 'song', 'year'], axis=1)
y_song = song['artist']

In [ ]:
X_song

In [ ]:
import re
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    text = re.sub(r"\d+", "", text, flags=re.UNICODE)
    return text.strip()

In [ ]:
X_song['lyrics'] = [preprocess_text(t) for t in X_song['lyrics']]
X_song['genre'] = pd.get_dummies(X_song['genre'])

In [ ]:
from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import sent_tokenize, word_tokenize
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(englishStemmer.stem(word))
        stem_sentence.append(" ")
    stem_sentence = [x for x in stem_sentence if x not in stop_words and len(x)>=3]
    return " ".join(stem_sentence)

In [ ]:
X_song['lyrics'] = [stemSentence(t) for t in X_song['lyrics']]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()
x = v.fit_transform(X_song['lyrics'])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
res = pd.concat([X_song, df1], axis=1)
X_song_data = res.drop(['lyrics'], axis=1)

In [ ]:
X_song_data

In [ ]:
x_trains, x_tests,y_trains, y_tests = train_test_split(X_song_data, y_song,train_size=0.75,random_state=1)
print(x_trains.shape[0],'x_train samples')
print(len(y_trains))
print(x_trains.shape)
print(x_tests.shape[0],'x_test samples')
print(len(y_tests))
print(x_tests.shape)

In [ ]:
scores_data_song = pd.DataFrame()
rc =np.random.seed(0)
for max_depth in range(2, 10):
    clf = tree.DecisionTreeClassifier(criterion='entropy', \
                max_depth=max_depth, random_state=rc, min_samples_leaf=3)
    clf.fit(x_trains, y_trains)
    train_score = clf.score(x_trains, y_trains)
    test_score = clf.score(x_tests, y_tests)
    mean_cross_val_score = cross_val_score(clf, x_trains, y_trains, cv=5).mean()
    temp_score_data = pd.DataFrame({'max_depth':[max_depth], 'train_score':[train_score], 'test_score':[test_score], 'cross_val_score':[mean_cross_val_score]})
    scores_data_song = scores_data_song.append(temp_score_data)

In [ ]:
scores_data_long_song = pd.melt(scores_data_song, id_vars =['max_depth'], value_vars =['train_score', 'test_score', 'cross_val_score'], var_name='set_type', value_name='score')

In [ ]:
sns.lineplot(x='max_depth', y='score', hue='set_type', data=scores_data_long_song)

In [ ]:
mod = tree.DecisionTreeClassifier(criterion='entropy', \
                max_depth=6, random_state=rc, min_samples_leaf=3)
mod.fit(x_trains, y_trains)

In [ ]:
predictions = mod.predict(x_tests)

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
precision_score(y_tests, predictions, average='micro')

In [ ]:
'''mod = DecisionTreeClassifier(criterion='entropy', \
                max_depth=10, random_state=np.random.seed(0), min_samples_leaf=3)
mod.fit(X_train, y_train)
predictions = mod.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')'''

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
clf = tree.DecisionTreeClassifier()
clf

In [ ]:
parametrs = {'criterion':['entropy', 'gini'], 'max_depth':range(1,30)}

In [ ]:
grid_search_cv_clf = GridSearchCV(clf, parametrs, cv=5)
grid_search_cv_clf.get_params()

In [ ]:
grid_search_cv_clf.fit(X_traint, y_traint)

In [ ]:
grid_search_cv_clf.best_params_

In [ ]:
best_clf = grid_search_cv_clf.best_estimator_
best_clf

In [ ]:
y_pred = best_clf.predict(X_testt)

In [ ]:
best_clf.score(X_testt, y_testt)

In [ ]:
print(precision_score(y_testt, y_pred, average='macro'))
print(recall_score(y_testt, y_pred, average='macro'))

In [ ]:
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_curve

In [ ]:
y_predicted_prob =  best_clf.predict_proba(X_testt)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_testt, y_predicted_prob[:, 1])
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
plt.plot(fpr, tpr, label = 'ROC curve (area=\0.2f)'% (roc_auc))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc='lower right')

In [ ]:
train_data_tree = pd.read_csv('../input/train-data-tree/train_data_tree.csv')

In [ ]:
train_data_tree

In [ ]:
X_tree = train_data_tree.drop(['num'], axis=1)
y_tree = train_data_tree['num']

In [ ]:
mod = tree.DecisionTreeClassifier(criterion='entropy')
param = {'min_samples_split': range(2,5), 'max_depth':range(2,30)}

In [ ]:
grid_search_cv_tree = GridSearchCV(mod, param, cv=5)
grid_search_cv_tree.get_params()

In [ ]:
grid_search_cv_tree.fit(X_tree, y_tree)
#grid_search_cv_tree.best_params_
best_tree = grid_search_cv_tree.best_estimator_
best_tree

In [ ]:
mod1=tree.DecisionTreeClassifier(criterion='entropy', max_depth=1)
mod1.fit(X_tree, y_tree)
mod1.score(X_tree, y_tree)

In [ ]:
tree.plot_tree(mod1, filled=True)

In [ ]:
mod1.tree_.n_node_samples

In [ ]:
mod1.tree_.impurity

In [ ]:
IG = 0.996 - (157*0.903 + 81*0.826)/238
IG

In [ ]:
users_data

In [ ]:
events_data.head()

In [ ]:
user_min_time = events_data.groupby('user_id', as_index=False).agg({'timestamp':'min'}).rename({'timestamp':'min_timestamp'}, axis=1)
user_min_time

In [ ]:
users_data = users_data.merge(user_min_time, how='outer')
users_data

In [ ]:
events_data['user_time'] = events_data.user_id.map(str) + '_' + events_data.timestamp.map(str)

In [ ]:
events_data.head()

In [ ]:
learning_time_htreshold = 3*24*60*60
learning_time_htreshold

In [ ]:
user_learning_time_htreshold = user_min_time.user_id.map(str) + '_' + (user_min_time.min_timestamp + learning_time_htreshold).map(str)

In [ ]:
user_learning_time_htreshold.head()

In [ ]:
user_min_time['user_learning_time_htreshold'] = user_learning_time_htreshold

In [ ]:
events_data = events_data.merge(user_min_time[['user_id', 'user_learning_time_htreshold']], how='outer')

In [ ]:
events_data 

In [ ]:
event_data_train = events_data[events_data.user_time <= events_data.user_learning_time_htreshold]
event_data_train

In [ ]:
event_data_train.groupby('user_id').day.nunique().max()

In [ ]:
submission_data['user_time'] = submission_data.user_id.map(str) + '_' + submission_data.timestamp.map(str)
submission_data = submission_data.merge(user_min_time[['user_id', 'user_learning_time_htreshold']], how='outer')
submission_data_train = submission_data[submission_data.user_time <= submission_data.user_learning_time_htreshold]
submission_data_train.groupby('user_id').day.nunique().max()

In [ ]:
X = submission_data_train.groupby('user_id').day.nunique().to_frame().rename(columns={'day':'days'}).reset_index()
X

In [ ]:
steps_tried = submission_data_train.groupby('user_id').step_id.nunique().to_frame().reset_index().rename(columns={'step_id':'steps_tried'})
steps_tried

In [ ]:
X = X.merge(steps_tried, on='user_id', how='outer')
X

In [ ]:
submission_data_train.pivot_table(index='user_id', columns='submission_status', values='step_id', aggfunc='count', fill_value=0).reset_index().head()

In [ ]:
X = X.merge(submission_data_train.pivot_table(index='user_id', columns='submission_status', values='step_id', aggfunc='count', fill_value=0).reset_index())
X['correct_ratio'] = X.correct / (X.correct + X.wrong)
X

In [ ]:
X = X.merge(event_data_train.pivot_table(index='user_id', columns='action', values='step_id', aggfunc='count', fill_value=0).reset_index()[['user_id', 'viewed']], how='outer')
X = X.fillna(0)
X

In [ ]:
X = X.merge(users_data[['user_id', 'passed_course', 'is_gone_user']], how='outer')
X

In [ ]:
X = X[~((X.is_gone_user == False) & (X.passed_course == False))]
X

In [ ]:
y = X.passed_course.astype(int)
y

In [ ]:
X.groupby(['passed_course', 'is_gone_user']).user_id.count()

In [ ]:
X = X.drop(['passed_course', 'is_gone_user'],axis=1)

In [ ]:
X = X.set_index(X.user_id)
X = X.drop('user_id', axis=1)
X

In [ ]:
X.hist(figsize=(10,6))

In [ ]:
mod2 = tree.DecisionTreeClassifier(criterion='entropy')
params = {'min_samples_split': range(2,5), 'max_depth':range(2,50)}
grid_search_cv_user = GridSearchCV(mod2, params, cv=5)
grid_search_cv_user.fit(X, y)
best_user = grid_search_cv_user.best_estimator_
best_user

In [ ]:
best_user.fit(X, y)
best_user.score(X, y)

In [ ]:
y.hist()

In [ ]:
heart = pd.read_csv('../input/heart-disease-uci/heart.csv')

In [ ]:
heart

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_heart = heart.drop(['target'], axis=1)
y_heart = heart['target']

In [ ]:

rf = RandomForestClassifier(10, max_depth=5, random_state =np.random.seed(0))

In [ ]:
rf.fit(X_heart, y_heart)

In [ ]:
imp = pd.DataFrame(rf.feature_importances_, index=X_heart.columns, columns=['importance'])
imp.sort_values('importance').plot(kind='barh', figsize=(12, 8))

In [ ]:
mush = pd.read_csv('../input/mushroom-classification/mushrooms.csv')

In [ ]:
mush

In [ ]:
'''Encoding categorical data¶
Label encoding

from sklearn.preprocessing import LabelEncoder
Encoder_X = LabelEncoder() 
for col in X.columns:
    X[col] = Encoder_X.fit_transform(X[col])
Encoder_y=LabelEncoder()
y = Encoder_y.fit_transform(y)'''

In [ ]:
train_mush = pd.read_csv('../input/train-mush/training_mush.csv.crdownload')

In [ ]:
train_mush

In [ ]:
train_mush = train_mush.dropna()

In [ ]:
X_mush = train_mush.drop('class', axis=1)
y_mush = train_mush['class']

In [ ]:
y_mush

In [ ]:

X_mush

In [ ]:
X_mush.shape

In [ ]:
pd.isnull(X_mush)

In [ ]:
r = RandomForestClassifier(random_state=0)

In [ ]:
params_r = {'n_estimators': range(10,50,10),'min_samples_split': range(2,9,2), 'max_depth':range(1,12,2), 'min_samples_leaf': range(1,7)}
grid_search_cv_r = GridSearchCV(r, params_r, cv=3, n_jobs=-1)
grid_search_cv_r.fit(X_mush, y_mush)
par = grid_search_cv_r.best_params_
best_r = grid_search_cv_r.best_estimator_
best_r

In [ ]:
grid_search_cv_r.best_params_

In [ ]:
imp_r = pd.DataFrame(best_r.feature_importances_, index=X_mush.columns, columns=['importance'])
imp_r.sort_values(by='importance', ascending=False)

In [ ]:
test_mush = pd.read_csv('../input/test-mush/testing_mush.csv.crdownload')
test_mush

In [ ]:
#testing = pd.read_csv('../input/testing/testing_y_mush.csv.zip')

In [ ]:
test_mush.isnull().sum().sum()

In [ ]:
test_mush = test_mush.fillna(0)

In [ ]:
best_r.fit(X_mush, y_mush)

In [ ]:
preds_mush = best_r.predict(test_mush)
rez_mush = pd.Series(preds_mush).value_counts()
rez_mush

In [ ]:
invasion = pd.read_csv('../input/invasion/invasion.csv')
invasion

In [ ]:
invasion.describe()

In [ ]:
invasion.isnull().sum().sum()

In [ ]:
X_invasion = invasion.drop('class', axis=1)
y_invasion = invasion['class']

In [ ]:
inv = RandomForestClassifier(random_state=0)

In [ ]:
params_inv = {'n_estimators': range(10,150,10),'min_samples_split': range(2,9,2), 'max_depth':range(1,20,2), 'min_samples_leaf': range(1,7)}
grid_search_cv_inv = GridSearchCV(inv, params_inv, cv=5, n_jobs=-1)
grid_search_cv_inv.fit(X_invasion, y_invasion)
grid_search_cv_inv.best_params_
best_inv = grid_search_cv_inv.best_estimator_
best_inv

In [ ]:
grid_search_cv_inv.best_params_

In [ ]:
best_inv = RandomForestClassifier(max_depth=3, n_estimators=10, min_samples_leaf=1, min_samples_split= 2, random_state=0)

In [ ]:
imp_inv = pd.DataFrame(best_inv.feature_importances_, index=X_invasion.columns, columns=['importance'])
imp_inv.sort_values(by='importance', ascending=False)

In [ ]:
best_inv.fit(X_invasion, y_invasion)

In [ ]:
%%timeit
df = pd.DataFrame(range(10000000))
%time df.apply(np.mean)


In [ ]:
%time df.apply('mean')


In [ ]:
%time df.describe().loc['mean']


In [ ]:
%time df.mean(axis=0)